In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import datetime
import requests
import pickle
import os
import re

In [2]:
def get_dir(dir_,symbol):
    if dir_[-1] == '/':
        to_dir_ = dir_ + symbol + '/10-K/'
    else:
        to_dir_ = dir_ + '/' + symbol + '/10-K/'
    to_run_list = [to_dir_ + doc for doc in os.listdir(to_dir_) if doc[-4:] == '.txt']   
    return to_run_list

In [3]:
def run_symbol(dir_:str,symbol:str):
    if (os.path.exists(dir_)):
        symbol_list = os.listdir(dir_)
        if symbol in symbol_list or symbol == 'all':
            to_run_list = []
            if symbol in symbol_list:
                to_run_list.extend(get_dir(dir_,symbol))
            elif symbol == 'all':
                for symbol in symbol_list:
                    if symbol[-4:] != '.ini':
                        to_run_list.extend(get_dir(dir_,symbol))
            else:
                print('Error : Invalid input symbol is Wrong!')
            print('System : Finish generate list!')
            return to_run_list
        else:
            print('Error : File <' + symbol + '> is not exist!')
    else:
        print('Error : File <' + dir_ + '> is not exist!')

In [4]:
def run_file(dir_:list):
    if len(dir_) == 0:
        print('Error : List direction has no item!')
        return 
    
    data_to_df = []
    try:
        for no,each_dir in enumerate(dir_):
            print('='*40)
            report = open(each_dir,'r')
            raw_data = bs(report,'html')
            text_location = get_items_table(raw_data)
            result = get_section_content(raw_data,text_location,None)
            year = each_dir.split('/')[-1].split('-')[1]
            symbol = each_dir.split('/')[-3]
            data_to_df.append([year,symbol,result])
            print('System : Complete summary {} of {}!'.format(no+1,len(dir_)))
            print('='*40)
    except Exception as e:
        print('Error : '+ e)

In [5]:
def check_exist(key_,val_):
    if key_ == None or key_ == '':
        return val_
    else:
        return key_

In [6]:
def create_file(dir_):
    try:
        if not os.path.exists(dir_):
            os.makedirs(dir_)
    except OSError:
        print('Error: Creating directory <' + str(dir_) + '>')

In [7]:
def request(link_:None):
    TARGET_LINK = 'https://www.sec.gov/Archives/edgar/data/916457/000091645719000016/cpn_10kx12312018.htm'
    link_ = check_exist(link_,TARGET_LINK)
    res = requests.get(link_).content.decode()
    raw_data = bs(res,'html') 
    return raw_data

In [8]:
def get_items_table(raw_data):
    text_location = {}
    part_count = 0
    begin_to_save = False
    
    try:
        for each_table in raw_data.find_all('table'):
            if each_table.find_all('font',text = 'TABLE OF CONTENTS') != '':
                for each_col in each_table.find_all('tr'):
                    if each_col.find('a') != None:
                        if 'PART' in each_col.find('a').get_text().split():
                            part_count+=1
                            begin_to_save = True
                            continue
                            if part_count >2:
                                break
                        if(begin_to_save):
                            text_location[each_col.find('a').get_text()] = each_col.find('a')['href']
        to_pickle(None,text_location)
        print('System : Get items table successfully!')
        return text_location
    except Exception as e:
        print('Error : {}'.format(e))

In [9]:
def get_section_content(raw_data,text_location,target_data_list:None):
    SCRAPE_LIST = ['Business','Risk Factors','Market for Registrant’s Common Equity, Related Stockholder Matters and Issuer Purchases of Equity Securities','Management’s Discussion and Analysis of Financial Condition and Results of Operations']
    target_data_list = check_exist(target_data_list,SCRAPE_LIST)
    
    result_ = []
    try:
        for key in target_data_list:
            section_data = raw_data.find('a',{ 'name' : text_location[ key].split('#')[1] }).parent.next_element.next_element
            text = section_data
            break_key = list(text_location.keys())[list(text_location.keys()).index(key)+1]
            break_point = text_location[break_key].split('#')[1]
            while(section_data):
                result_.append(text.get_text())
                text = text.next_sibling
                try:
                    if text.find('a')['name'] == break_point:
                        break
                except:
                    pass
        return result_
    except Exception as e:
        print('Error : {}'.format(e))

In [10]:
def to_pickle(target_path:None,content):
    TARGET_PATH = 'C:/Users/sefx5/Desktop/'
    target_path = check_exist(target_path,TARGET_PATH)
    
    if content:
        file_name = target_path + '10-K_' + str(datetime.date.today()) + '.pkl'
        with open(file_name,'wb') as f:
            pickle.dump(content,f)
            f.close()
        print('System : Covert pickle successfully! File name <' + file_name + '>')
    else:
        print('Error : Failed to save data! EMPTY CONTENT')

In [11]:
def to_csv(data_to_csv):
    data_to_csv.to_csv('10-k_summary.csv',index = False,header = True,encoding='utf-8')
    print('System : Covert csv successfully! File name <10-k_summary.csv>')

In [12]:
def read_pickle(target_path:None,file_name):
    TARGET_PATH = 'C:/Users/sefx5/Desktop/'
    target_path = check_exist(target_path,TARGET_PATH)

    try:
        with open(target_path + file_name,'rb') as f:
            content = pickle.load(f)
            return content
            print('System : Read pickle successfully!')
    except Exception as e:
        print('Error : Failed to read file!')

# Running Data 

In [ ]:
dir_ = 'G:/My Drive/Fintech_Nomura/Database/10-K Data/sec_edgar_filings/'
direction_list = run_symbol(dir_,'all')
run_file(direction_list)

System : Finish generate list!
Error : 'href'
Error : 'NoneType' object is not subscriptable
System : Complete summary 1 of 236!
System : Covert pickle successfully! File name <C:/Users/sefx5/Desktop/10-K_2020-05-11.pkl>
System : Get items table successfully!
Error : 'NoneType' object has no attribute 'parent'
System : Complete summary 2 of 236!
Error : 'href'
Error : 'NoneType' object is not subscriptable
System : Complete summary 3 of 236!
System : Covert pickle successfully! File name <C:/Users/sefx5/Desktop/10-K_2020-05-11.pkl>
System : Get items table successfully!
Error : 'NoneType' object has no attribute 'parent'
System : Complete summary 4 of 236!
Error : 'href'
Error : 'NoneType' object is not subscriptable
System : Complete summary 5 of 236!
Error : Failed to save data! EMPTY CONTENT
System : Get items table successfully!
Error : 'Business'
System : Complete summary 6 of 236!
Error : Failed to save data! EMPTY CONTENT
System : Get items table successfully!
Error : 'Business'

# Test Data

In [ ]:
raw_data = request(None)
text_location = get_items_table(raw_data)
result = get_section_content(raw_data,text_location,None)

In [ ]:
to_pickle(None,result)

In [ ]:
read_pickle('C:/Users/sefx5/Desktop/','10-K_2020-05-11.pkl')

In [ ]:
x = open('0001091667-16-000396.txt','r',encoding='utf8')